# Running on new images
This notebook will walk you step by step through the process of using a pre-trained model to detect traffic signs in an image.

# Imports

In [32]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import os
import tensorflow as tf


from matplotlib import pyplot as plt
import glob as glob
import random
import pickle

import sys
import cv2
import csv

%matplotlib inline

# tf.executing_eagerly()

In [33]:
score_threshold = 0.3
np.random.seed(1)

# Environment setup

In [34]:


sys.path.append('../models/research')  # Replace with the path to TensorFlow Object Detection API
sys.path.append('../darkflow')  # Replace with the path to Darkflow

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


## Model preparation 

# Tensorflow Object Detection API

In [35]:
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous'
# MODEL_NAME = 'faster_rcnn_resnet_101'
# MODEL_NAME = 'faster_rcnn_resnet50'
MODEL_NAME = 'faster_rcnn_inception_v2'
# MODEL_NAME = 'rfcn_resnet101'
# MODEL_NAME = 'ssd_inception_v2'
# MODEL_NAME = 'ssd_mobilenet_v1'

In [36]:
# Path to frozen detection graph. This is the actual model that is used for the traffic sign detection.
MODEL_PATH = os.path.join('models', MODEL_NAME)
PATH_TO_CKPT = os.path.join(MODEL_PATH,'inference_graph/frozen_inference_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('gtsdb_data', 'gtsdb3_label_map.pbtxt')

NUM_CLASSES = 3


## Load a (frozen) Tensorflow model into memory

In [37]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `2`, we know that this corresponds to `mandatory`.

In [38]:
PATH_TO_LABELS = r'D:\my_project\traffic-sign-detection\gtsdb_data\gtsdb3_label_map.pbtxt'

# Directly reading the file for troubleshooting
try:
    with open(PATH_TO_LABELS, 'rb') as f:
        print("Direct read:", f.read())
except Exception as e:
    print("File read error:", str(e))

# Your existing code
print('1')
try:
    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
except Exception as e:
    print("Error during label map load:", str(e))
print('2')
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
print('3')
category_index = label_map_util.create_category_index(categories)
print(label_map)



Direct read: b"item {\n  id: 1\n  name: 'prohibitory'\n}\n\nitem {\n  id: 2\n  name: 'mandatory'\n}\n\nitem {\n  id: 3\n  name: 'danger'\n}\n\n"
1
2
3
item {
  name: "prohibitory"
  id: 1
}
item {
  name: "mandatory"
  id: 2
}
item {
  name: "danger"
  id: 3
}



## Helper code

In [39]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    #return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
    return np.array(image.getdata()).reshape((im_height, im_width, 3))

## Detection

In [40]:

PATH_TO_TEST_IMAGES_DIR =  r'D:\my_project\traffic-sign-detection\test_images'

TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))

# Size, in inches, of the output images.
IMAGE_SIZE = (20, 20)

In [41]:

def append_results(results, image_name, iteration, class_name, score, box, sum_diff, max_diff):
    results.append({
        'image_name': image_name,
        'iteration': iteration + 1,
        'class': class_name,
        'score': float(score),
        'box': box,
        'total_perturbation': sum_diff,
        'max_perturbation': max_diff
    })

def save_results_to_csv(results, image_index, save_dir, algorithm_number):
    csv_filename = f"adv_alg{algorithm_number}_results.csv"
    csv_path = os.path.join(save_dir, csv_filename)
    
    # Check if file exists to decide whether to write headers or not
    file_exists = os.path.isfile(csv_path)
    
    with open(csv_path, 'a', newline='') as csvfile:
        # Include 'image_index' in the fieldnames
        fieldnames = ['image_index', 'image_name', 'iteration', 'class', 'score', 'box', 'total_perturbation', 'max_perturbation']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # If the file does not exist, write the header
        if not file_exists:
            writer.writeheader()
        
        for result in results:
            # Include image_index if you want to track which image the results belong to
            result['image_index'] = image_index + 1
            writer.writerow(result)


# Functions

In [42]:
def region_downsample(image_region, factor):
    return cv2.resize(image_region, None, fx=factor, fy=factor, interpolation=cv2.INTER_LINEAR)

def region_upsample(image_region, target_shape):
    return cv2.resize(image_region, (target_shape[1], target_shape[0]), interpolation=cv2.INTER_LINEAR)

def compute_loss(sess, image_np_expanded, detection_graph):
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    
    (scores, classes) = sess.run([scores, classes], feed_dict={image_tensor: image_np_expanded})
    max_score_index = np.argmax(scores)
    score = scores[0][max_score_index]
    approx_logits = np.log(score / (1 - score))
    return approx_logits

def approximate_gradient(sess, image_np_expanded, detection_graph, h, num_samples):
    original_loss = compute_loss(sess, image_np_expanded, detection_graph)
    grad_approx = np.zeros_like(image_np_expanded).astype(np.float32)
    
    total_pixels = np.prod(image_np_expanded.shape)
    num_samples = min(num_samples, total_pixels)  # Adjust the num_samples value if total_pixels is less than NUM_SAMPLES
    indices = np.random.choice(total_pixels, num_samples, replace=False)  # Randomly selecting pixel indices from the entire image
        
    for flat_idx in indices:
        idx = np.unravel_index(flat_idx, image_np_expanded.shape)
        perturb = np.zeros_like(image_np_expanded)
        perturb[idx] = h
        perturbed_image = image_np_expanded + perturb
        perturbed_loss = compute_loss(sess, perturbed_image, detection_graph)
        grad_approx[idx] = (perturbed_loss - original_loss) / h 

    return grad_approx

def save_adversarial_image(image, idx, iteration, save_dir, algorithm_number):
    image_name = f"alg{algorithm_number}_{idx + 1:03d}_itr{iteration + 1}.jpg"
    image_save_path = os.path.join(save_dir, image_name)
    plt.imsave(image_save_path, image)
    return image_name

def save_adversarial_image_with_boxes(image, idx, iteration, save_dir, algorithm_number, suffix="box"):
    image_with_boxes_name = f"boxalg{algorithm_number}_{idx + 1:03d}_iter{iteration + 1}.jpg"
    image_with_boxes_save_path = os.path.join(save_dir, image_with_boxes_name)
    plt.imsave(image_with_boxes_save_path, image)
    return image_with_boxes_name

def process_bounding_boxes(sess, adv_image, boxes_tensor, scores_tensor, classes_tensor, num_detections_tensor, image_tensor, score_threshold, buffer_factor, original_shape, factor, num_samples, h):
    grad_approx = np.zeros_like(adv_image).astype(np.float32)
    
    # Fetch the original detections to compute bounding boxes
    (initial_boxes, initial_scores, _, _) = sess.run(
        [boxes_tensor, scores_tensor, classes_tensor, num_detections_tensor],
        feed_dict={image_tensor: np.expand_dims(adv_image, axis=0)}
    )
    initial_boxes = np.squeeze(initial_boxes)
    initial_scores = np.squeeze(initial_scores)

    valid_indices = np.where(initial_scores > score_threshold)[0]  # Filter detections based on threshold

    # For each valid detection
    for valid_idx in valid_indices:
        valid_box = initial_boxes[valid_idx]
        ymin, xmin, ymax, xmax = valid_box
        height, width = ymax - ymin, xmax - xmin

        ymin = max(0, ymin - buffer_factor * height)
        xmin = max(0, xmin - buffer_factor * width)
        ymax = min(1, ymax + buffer_factor * height)
        xmax = min(1, xmax + buffer_factor * width)

        ymin, xmin, ymax, xmax = int(ymin * original_shape[0]), int(xmin * original_shape[1]), int(ymax * original_shape[0]), int(xmax * original_shape[1])

        if ymax - ymin > 0 and xmax - xmin > 0:
            region = adv_image[ymin:ymax, xmin:xmax]
            region_low_res = region_downsample(region, factor)
            region_low_res_expanded = np.expand_dims(region_low_res, axis=0)

            # Compute the approximate gradient on low-res region
            region_grad_approx_low_res = approximate_gradient(sess, region_low_res_expanded, detection_graph, h, num_samples)
            # Upsample gradient back to original resolution of that region
            region_grad_approx = region_upsample(region_grad_approx_low_res[0], region.shape)

            grad_approx[ymin:ymax, xmin:xmax] = region_grad_approx

    return grad_approx


# Gradient Based Method (adversarial pattern only generated in and around bounding boxes)

In [47]:
# Tuning parameters
FACTOR = 0.2
H = 10
NUM_SAMPLES = 56
NUM_ITERATIONS = 10
EPS = 10000
BUFFER_FACTOR =2
algorithm_number = 21  # Adjust this for different algorithms


# Define the directory for saving images and results
SAVE_DIR = '200_images_21'
os.makedirs(SAVE_DIR, exist_ok=True)

# Main execution
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            original_shape = image_np.shape

            adv_image = image_np.copy()
            last_adv_image = np.copy(adv_image)  # To calculate the difference per iteration
            results = []

            # Iteration based on images
            for i in range(NUM_ITERATIONS):
                grad_approx = process_bounding_boxes(
                    sess, adv_image, boxes, scores, classes, num_detections, image_tensor, 0.5, 
                    BUFFER_FACTOR, original_shape, FACTOR, NUM_SAMPLES, H
                )

                perturbed_adv_image = adv_image - EPS * grad_approx  # Perturb the image

                # Calculate the total and max perturbation before clipping
                per_box_diff = EPS * grad_approx
                sum_diff = np.sum(np.abs(per_box_diff), axis=(0, 1, 2))
                max_diff = np.max(np.abs(per_box_diff), axis=(0, 1, 2))

                # Print out the desired information after the perturbation and before clipping
                print(f"Test Image {idx + 1}, Iteration {i + 1}:")
                print(f"Sum_diff: {sum_diff}")
                print(f"Max_diff: {max_diff}")

                # Initialize a flag to check if any scores are above the threshold
                any_above_threshold = False

                # Update the last_adv_image with the perturbed values before clipping for next iteration's comparison
                last_adv_image = perturbed_adv_image.copy()

                # Now apply clipping to keep pixel values valid for visualizing and further processing
                adv_image = np.clip(perturbed_adv_image, 0, 255).astype(np.uint8)

                # Save the adversarial image and get the image name
                image_name = save_adversarial_image(adv_image, idx, i, SAVE_DIR, algorithm_number)

                adv_image_expanded = np.expand_dims(adv_image, axis=0)
                (iter_boxes, iter_scores, iter_classes, iter_num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: adv_image_expanded}
                )
                
                # Save perturbation details per box
                for j, score in enumerate(np.squeeze(iter_scores)):
                    if score > score_threshold:
                        any_above_threshold = True
                        class_id = int(np.squeeze(iter_classes)[j])
                        class_name = category_index[class_id]['name']
                        box = np.squeeze(iter_boxes)[j].tolist()
                        append_results(results, image_name, i, class_name, score, box, sum_diff, max_diff)
                        print(f"Detected class: {class_name}, Confidence Score: {score:.8f}")

                # If no scores were above threshold, record the max and sum diff
                if not any_above_threshold:
                    append_results(results, image_name, i, "no_detection", 0, [0, 0, 0, 0], sum_diff, max_diff)


                # Visualization of the results
                adv_image_visual = adv_image.copy()
                # vis_util.visualize_boxes_and_labels_on_image_array(
                #     adv_image_visual, 
                #     np.squeeze(iter_boxes),
                #     np.squeeze(iter_classes).astype(np.int32),
                #     np.squeeze(iter_scores),
                #     category_index,
                #     use_normalized_coordinates=True,
                #     line_thickness=6,
                #     min_score_thresh=0.5
                # )

                # Save the visualized image with boxes
                save_adversarial_image_with_boxes(adv_image_visual, idx, i, SAVE_DIR, algorithm_number)


                # plt.figure(figsize=IMAGE_SIZE)
                # plt.axis('off')
                # plt.imshow(adv_image_visual) 
                # plt.title(f"Iteration {i + 1} Detection Results")
                # plt.show()
                
            # After all iterations for a single image, save the results
            save_results_to_csv(results, idx, SAVE_DIR, algorithm_number)

Test Image 1, Iteration 1:
Sum_diff: 473661.1875
Max_diff: 5735.77001953125
Detected class: mandatory, Confidence Score: 0.99997258
Test Image 1, Iteration 2:
Sum_diff: 16541.390625
Max_diff: 272.2866516113281
Detected class: mandatory, Confidence Score: 0.99980766
Test Image 1, Iteration 3:
Sum_diff: 20578.69140625
Max_diff: 452.1943664550781
Test Image 1, Iteration 4:
Sum_diff: 0.0
Max_diff: 0.0
Test Image 1, Iteration 5:
Sum_diff: 0.0
Max_diff: 0.0
Test Image 1, Iteration 6:
Sum_diff: 0.0
Max_diff: 0.0
Test Image 1, Iteration 7:
Sum_diff: 0.0
Max_diff: 0.0
Test Image 1, Iteration 8:
Sum_diff: 0.0
Max_diff: 0.0
Test Image 1, Iteration 9:
Sum_diff: 0.0
Max_diff: 0.0
Test Image 1, Iteration 10:
Sum_diff: 0.0
Max_diff: 0.0
Test Image 2, Iteration 1:
Sum_diff: 151638.46875
Max_diff: 4699.6357421875
Detected class: danger, Confidence Score: 0.99996793
Test Image 2, Iteration 2:
Sum_diff: 25668.2421875
Max_diff: 427.2394714355469
Detected class: danger, Confidence Score: 0.99996758
Test Im

# Projected Gradient Descent Method (adversarial pattern only generated in and around bounding boxes)

In [44]:
def project_perturbation(perturbation, epsilon, p_norm='inf'):
    """
    Project the perturbation onto an epsilon ball.
    """
    if p_norm == 'inf':
        return np.clip(perturbation, -epsilon, epsilon)
    elif p_norm == 2:
        norm = np.linalg.norm(perturbation)
        if norm > epsilon:
            return perturbation * (epsilon / norm)
        return perturbation
    else:
        raise NotImplementedError("Only L-inf and L-2 norms are supported")
    

In [45]:
# Tuning parameters
FACTOR = 0.2
H = 10
NUM_SAMPLES = 20
NUM_ITERATIONS = 10
EPS = 1000
BUFFER_FACTOR =1
algorithm_number = 19  # Adjust this for different algorithms

epsilon = 1000

# Define the directory for saving images and results
SAVE_DIR = '200_images_19'
os.makedirs(SAVE_DIR, exist_ok=True)

# Main execution
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            original_shape = image_np.shape

            adv_image = image_np.copy()
            last_adv_image = np.copy(adv_image)  # To calculate the difference per iteration
            results = []

            # Iteration based on images
            for i in range(NUM_ITERATIONS):
                grad_approx = process_bounding_boxes(
                    sess, adv_image, boxes, scores, classes, num_detections, image_tensor, 0.5, 
                    BUFFER_FACTOR, original_shape, FACTOR, NUM_SAMPLES, H
                )

                # Project the perturbation onto the epsilon ball
                perturbation = EPS * grad_approx
                perturbation = project_perturbation(perturbation, epsilon)
                # perturbation = project_perturbation(perturbation, epsilon, p_norm= 2)

                # adv_image = adv_image.astype(np.float32)
                perturbed_adv_image = adv_image - perturbation 

                # Calculate the total and max perturbation before clipping
                sum_diff = np.sum(np.abs(perturbation), axis=(0, 1, 2))
                max_diff = np.max(np.abs(perturbation), axis=(0, 1, 2))

                # Print out the desired information after the perturbation and before clipping
                print(f"Test Image {idx + 1}, Iteration {i + 1}:")
                print(f"Sum_diff: {sum_diff}")
                print(f"Max_diff: {max_diff}")

                # Initialize a flag to check if any scores are above the threshold
                any_above_threshold = False

                # Now apply clipping to keep pixel values valid for visualizing and further processing
                adv_image = np.clip(perturbed_adv_image, 0, 255).astype(np.uint8)

                # Update the last_adv_image with the perturbed values before clipping for next iteration's comparison
                last_adv_image = perturbed_adv_image.copy()


                # Save the adversarial image and get the image name
                image_name = save_adversarial_image(adv_image, idx, i, SAVE_DIR, algorithm_number)

                adv_image_expanded = np.expand_dims(adv_image, axis=0)
                (iter_boxes, iter_scores, iter_classes, iter_num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: adv_image_expanded}
                )
                
                # Save perturbation details per box
                for j, score in enumerate(np.squeeze(iter_scores)):
                    if score > score_threshold:
                        any_above_threshold = True
                        class_id = int(np.squeeze(iter_classes)[j])
                        class_name = category_index[class_id]['name']
                        box = np.squeeze(iter_boxes)[j].tolist()
                        append_results(results, image_name, i, class_name, score, box, sum_diff, max_diff)
                        print(f"Detected class: {class_name}, Confidence Score: {score:.8f}")

                # If no scores were above threshold, record the max and sum diff
                if not any_above_threshold:
                    append_results(results, image_name, i, "no_detection", 0, [0, 0, 0, 0], sum_diff, max_diff)


                # Visualization of the results
                adv_image_visual = adv_image.copy()
                vis_util.visualize_boxes_and_labels_on_image_array(
                    adv_image_visual, 
                    np.squeeze(iter_boxes),
                    np.squeeze(iter_classes).astype(np.int32),
                    np.squeeze(iter_scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=6,
                    min_score_thresh=0.5
                )

                # Save the visualized image with boxes
                save_adversarial_image_with_boxes(adv_image_visual, idx, i, SAVE_DIR, algorithm_number)

                # plt.figure(figsize=IMAGE_SIZE)
                # plt.axis('off')
                # plt.imshow(adv_image_visual) 
                # plt.title(f"Iteration {i + 1} Detection Results")
                # plt.show()
                
            # After all iterations for a single image, save the results
            save_results_to_csv(results, idx, SAVE_DIR, algorithm_number)


Test Image 1, Iteration 1:
Sum_diff: 1472.7047119140625
Max_diff: 17.16919708251953
Detected class: mandatory, Confidence Score: 0.99993932
Test Image 1, Iteration 2:
Sum_diff: 570.6012573242188
Max_diff: 6.554397106170654
Detected class: mandatory, Confidence Score: 0.99993300
Test Image 1, Iteration 3:
Sum_diff: 371.0657043457031
Max_diff: 5.150062561035156
Detected class: mandatory, Confidence Score: 0.99994075
Test Image 1, Iteration 4:
Sum_diff: 1511.2862548828125
Max_diff: 12.53036117553711
Detected class: mandatory, Confidence Score: 0.99994886
Test Image 1, Iteration 5:
Sum_diff: 1025.06103515625
Max_diff: 13.103368759155273
Detected class: mandatory, Confidence Score: 0.99994969
Test Image 1, Iteration 6:
Sum_diff: 1250.087646484375
Max_diff: 10.747852325439453
Detected class: mandatory, Confidence Score: 0.99995124
Test Image 1, Iteration 7:
Sum_diff: 750.3681640625
Max_diff: 8.15400505065918
Detected class: mandatory, Confidence Score: 0.99995244
Test Image 1, Iteration 8:
S

In [46]:
# Tuning parameters
FACTOR = 0.2
H = 10
NUM_SAMPLES = 20
NUM_ITERATIONS = 10
EPS = 1000
BUFFER_FACTOR =1
algorithm_number = 20  # Adjust this for different algorithms

epsilon = 1000

# Define the directory for saving images and results
SAVE_DIR = '200_images_20'
os.makedirs(SAVE_DIR, exist_ok=True)

# Main execution
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            original_shape = image_np.shape

            adv_image = image_np.copy()
            last_adv_image = np.copy(adv_image)  # To calculate the difference per iteration
            results = []

            # Iteration based on images
            for i in range(NUM_ITERATIONS):
                grad_approx = process_bounding_boxes(
                    sess, adv_image, boxes, scores, classes, num_detections, image_tensor, 0.5, 
                    BUFFER_FACTOR, original_shape, FACTOR, NUM_SAMPLES, H
                )

                # Project the perturbation onto the epsilon ball
                perturbation = EPS * grad_approx
                # perturbation = project_perturbation(perturbation, epsilon)
                perturbation = project_perturbation(perturbation, epsilon, p_norm= 2)

                # adv_image = adv_image.astype(np.float32)
                perturbed_adv_image = adv_image - perturbation 

                # Calculate the total and max perturbation before clipping
                sum_diff = np.sum(np.abs(perturbation), axis=(0, 1, 2))
                max_diff = np.max(np.abs(perturbation), axis=(0, 1, 2))

                # Print out the desired information after the perturbation and before clipping
                print(f"Test Image {idx + 1}, Iteration {i + 1}:")
                print(f"Sum_diff: {sum_diff}")
                print(f"Max_diff: {max_diff}")

                # Initialize a flag to check if any scores are above the threshold
                any_above_threshold = False

                # Now apply clipping to keep pixel values valid for visualizing and further processing
                adv_image = np.clip(perturbed_adv_image, 0, 255).astype(np.uint8)

                # Update the last_adv_image with the perturbed values before clipping for next iteration's comparison
                last_adv_image = perturbed_adv_image.copy()


                # Save the adversarial image and get the image name
                image_name = save_adversarial_image(adv_image, idx, i, SAVE_DIR, algorithm_number)

                adv_image_expanded = np.expand_dims(adv_image, axis=0)
                (iter_boxes, iter_scores, iter_classes, iter_num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: adv_image_expanded}
                )
                
                # Save perturbation details per box
                for j, score in enumerate(np.squeeze(iter_scores)):
                    if score > score_threshold:
                        any_above_threshold = True
                        class_id = int(np.squeeze(iter_classes)[j])
                        class_name = category_index[class_id]['name']
                        box = np.squeeze(iter_boxes)[j].tolist()
                        append_results(results, image_name, i, class_name, score, box, sum_diff, max_diff)
                        print(f"Detected class: {class_name}, Confidence Score: {score:.8f}")

                # If no scores were above threshold, record the max and sum diff
                if not any_above_threshold:
                    append_results(results, image_name, i, "no_detection", 0, [0, 0, 0, 0], sum_diff, max_diff)


                # Visualization of the results
                adv_image_visual = adv_image.copy()
                vis_util.visualize_boxes_and_labels_on_image_array(
                    adv_image_visual, 
                    np.squeeze(iter_boxes),
                    np.squeeze(iter_classes).astype(np.int32),
                    np.squeeze(iter_scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=6,
                    min_score_thresh=0.5
                )

                # Save the visualized image with boxes
                save_adversarial_image_with_boxes(adv_image_visual, idx, i, SAVE_DIR, algorithm_number)

                # plt.figure(figsize=IMAGE_SIZE)
                # plt.axis('off')
                # plt.imshow(adv_image_visual) 
                # plt.title(f"Iteration {i + 1} Detection Results")
                # plt.show()
                
            # After all iterations for a single image, save the results
            save_results_to_csv(results, idx, SAVE_DIR, algorithm_number)


Test Image 1, Iteration 1:
Sum_diff: 1047.2716064453125
Max_diff: 11.176250457763672
Detected class: mandatory, Confidence Score: 0.99996209
Test Image 1, Iteration 2:
Sum_diff: 1408.35693359375
Max_diff: 17.66124725341797
Detected class: mandatory, Confidence Score: 0.99992883
Test Image 1, Iteration 3:
Sum_diff: 595.2852172851562
Max_diff: 8.035539627075195
Detected class: mandatory, Confidence Score: 0.99993038
Test Image 1, Iteration 4:
Sum_diff: 814.9263916015625
Max_diff: 6.939693927764893
Detected class: mandatory, Confidence Score: 0.99992740
Test Image 1, Iteration 5:
Sum_diff: 524.6478271484375
Max_diff: 8.266984939575195
Detected class: mandatory, Confidence Score: 0.99991727
Test Image 1, Iteration 6:
Sum_diff: 652.913330078125
Max_diff: 9.63265323638916
Detected class: mandatory, Confidence Score: 0.99992120
Test Image 1, Iteration 7:
Sum_diff: 751.710693359375
Max_diff: 7.479734897613525
Detected class: mandatory, Confidence Score: 0.99991620
Test Image 1, Iteration 8:
Su